In [1]:
from datetime import datetime, timedelta, timezone
import logging
import sys
import pandas as pd
import ccxt
import asyncio

In [2]:
async def async_wrap(func):
    # Wrapper to run the function in a thread (non-blocking)
    async def wrapper(*args, **kwargs):
        return await asyncio.to_thread(func, *args, **kwargs)
    return wrapper


def to_csv(data, filename, mode='w'):
    data.to_csv(filename, mode=mode)

async def vol_index_history(currency, exchange,
                            end=(datetime.now(tz=timezone.utc).replace(minute=0, second=0, microsecond=0)),
                            start=(
                                  datetime.now(tz=timezone.utc).replace(minute=0, second=0, microsecond=0)) - timedelta(
                                days=30),
                            timeframe='1h') -> pd.DataFrame:
    max_mark_data = int(1000)
    resolution = int(exchange.describe()['timeframes'][timeframe]) * 60

    e = int(end.timestamp())
    s = int(start.timestamp())
    f = int(max_mark_data * resolution)
    start_times = [s + k * f for k in range(1 + int((e - s) / f)) if s + k * f < e]

    logging.info(f'calling {sys._getframe(1).f_code.co_name} {len(start_times)} times')
    vol_index_coros = [
        exchange.publicGetGetVolatilityIndexData(params={'currency': currency, 'resolution': resolution,
                                                         'start_timestamp': start_time * 1000,
                                                         'end_timestamp': (start_time + f - int(resolution)) * 1000})
        for start_time in start_times]
    
    vol_index = [y for x in vol_index_coros for y in x['result']['data']]

    column_names = ['t', 'vol_index_o', 'vol_index_h', 'vol_index_l', 'vol_index_c']
    if not vol_index:
        return pd.DataFrame()

    ###### spot
    data = pd.DataFrame(columns=column_names, data=vol_index, dtype=float).astype(dtype={'t': 'int64'}).set_index(
        't') / 100
    data.index = [datetime.fromtimestamp(x / 1000, timezone.utc) for x in data.index]
    data = data[~data.index.duplicated()].sort_index()
    
    data["vol_index"] = (data["vol_index_o"] + data["vol_index_c"]) / 2
    to_csv(data, currency + '_volIndex.csv', mode='w')

    return data

In [3]:
exchange = ccxt.deribit()
await vol_index_history('ETH', exchange, start=datetime(2024, 1, 1, tzinfo=timezone.utc), end=datetime(2024, 9, 1, tzinfo=timezone.utc))

,vol_index_o,vol_index_h,vol_index_l,vol_index_c,vol_index
2024-01-01 00:00:00+00:00,0.7015,0.7075,0.7014,0.7015,0.70150
2024-01-01 01:00:00+00:00,0.7015,0.7015,0.6918,0.6918,0.69665
2024-01-01 02:00:00+00:00,0.6918,0.6951,0.6899,0.6927,0.69225
2024-01-01 03:00:00+00:00,0.6928,0.6946,0.6920,0.6931,0.69295
2024-01-01 04:00:00+00:00,0.6931,0.6940,0.6919,0.6940,0.69355
...,...,...,...,...,...
2024-09-06 19:00:00+00:00,0.7264,0.7288,0.7249,0.7282,0.72730
2024-09-06 20:00:00+00:00,0.7282,0.7404,0.7269,0.7404,0.73430
2024-09-06 21:00:00+00:00,0.7404,0.7426,0.7354,0.7385,0.73945
2024-09-06 22:00:00+00:00,0.7385,0.7385,0.7328,0.7328,0.73565
